In [695]:
import pandas as pd
import numpy as np

In [696]:
pr = pd.read_excel('C:/Users/ScMa9004/Documents/Итоги визита/314/Panel Russia_for 314.xlsx',
                   sheet_name = 'Panel', 
                   skiprows = 2)

In [697]:
rules = pd.read_excel('C:/Users/ScMa9004/Documents/Panel/NEW PANEL/FMCG_CIG_BEER.xlsx',
                      sheet_name = 'index + mcellname')

In [698]:
arc = pd.read_excel('C:/Users/ScMa9004/Documents/All Russia cities 01.01.2018.xlsx',
                    sheet_name = 'All_Russia_cities', 
                    skiprows = 1)

In [699]:
uo200 = pd.read_excel('C:/Users/ScMa9004/Documents/Universe Over 200.xls', 
                      sheet_name = 'Towns&Villages', 
                      skiprows = 1)

In [700]:
df = pd.read_excel('C:/Users/ScMa9004/Downloads/panel_report_00001_14.01.2020.xlsx',
                   sheet_name = 'panel_report_00001_14.01.2020')

In [852]:
newp = pd.read_excel('C:/Users/ScMa9004/Documents/Panel/NEW PANEL/NEW_PANEL_21-Oct_29_10_for_cost.xlsx',
                   sheet_name = 'Panel_Urban')

In [702]:
# prepare file

wrk = arc[['City Code',
            '# of Federal Subject', 
            'pgr1']].rename(columns = {'City Code': 'ccode',
                                                               '# of Federal Subject': 'Sub_id'})

pr = pd.merge(pr, wrk, on = 'ccode', how = 'left')
#pr = pd.merge(pr, np[['region_normalny', 'ccode']], on = 'ccode', how = 'left')

pr = pr.rename(columns = {'act stand': 'AP_FMCG', 
                          'act cig ': 'AP_CIG', 
                          'act beer ': 'AP_BEER'})

pr.loc[pr['hip'] == 1, 'hip'] = 'Y'
pr.loc[pr['hip'] == 0, 'hip'] = 'N'

pr = pr[['ccode', 'type', 'region_da', 'pgr1', 'ttr', 'hip', 'Sub_id', 'sub', 'AP_FMCG', 'AP_CIG', 'AP_BEER', 'ops']]

pr = pd.merge(pr, newp[['region_normalny', 'ccode']].drop_duplicates(), on = 'ccode', how = 'left')

pr['MCELL_ID_FMCG'] = np.nan
pr['MCELL_ID_CIG'] = np.nan
pr['MCELL_ID_BEER'] = np.nan
pr['MCELL_ID_EXXON'] = np.nan

In [704]:
def parser(mcell):
    if mcell.split('-')[0] in ['CNT', 'FST', 'VOL', 'URL', 'NWT', 'STH', 'MSO', 'SIB']:
        area = [mcell.split('-')[0]]
        if 'RR' in mcell.split('-')[-1]:
            ccodes = []
            subreg = list(map(int, mcell.split('-')[1].split('|')))
            types = ['OSU', 'OLA', 'OME', 'OSM', 'OPV', 'OMM', 'OMS']
            pgr = list(mcell.split('-')[2].split('|'))
            ds = list(mcell.split('-')[3].split('|'))
            hip = []
            ndx = [mcell.split('-')[-1]]
            NO_city = []
            ttr = ['MT|TT']
        else:
            ccodes = []
            subreg = list(map(int, mcell.split('-')[1].split('|')))
            if ('IML' in mcell.split('-')[2]) | ('IMS' in mcell.split('-')[2]):
                types = ['OKS', 'ONS', 'OPV', 'OSM']
                hip = [mcell.split('-')[2]]
            else:
                types = list(mcell.split('-')[2].split('|'))
                hip = []
            pgr = list(map(int, mcell.split('-')[3].split('|')))
            ttr = list(mcell.split('-')[4].split('|'))
            try:
                NO_city = list(map(int, mcell.split('-')[5].replace('(', '').replace(')', '').split('&')))
            except:
                NO_city = []
            ndx = [mcell.split('-')[-1]]
            ds = []
    else:
        if '|' in mcell.split('-')[0]:
            area = [mcell.split('-')[0] + ' ' + mcell.split('-')[1]]
        else:
            area = [mcell.split('-')[1]]
        ccodes = list(map(int, mcell.split('-')[0].split('|')))
        subreg = []
        hip = []
        if ('IML' in mcell.split('-')[2]) | ('IMS' in mcell.split('-')[2]):
            types = ['OKS', 'ONS', 'OPV', 'OSM']
            hip = [mcell.split('-')[2]]
        else:
            types = list(mcell.split('-')[2].split('|'))
        if ('Y' in mcell.split('-')[3]) | ('N' in mcell.split('-')[3]):
            ttr = ['TT']
            hip = list(mcell.split('-')[3])
        else:
            ttr = list(mcell.split('-')[3].split('|'))
        NO_city = []
        pgr = []
        ndx = [mcell.split('-')[-1]]
        ds = []
    return(area, ccodes, subreg, types, pgr, ttr, NO_city, hip, ds, ndx)

In [705]:
# mcell id for panel
for i, j in rules.iterrows():
    conditions = parser(j['new_mcell_name'])
    if conditions[1]:
        c1 = pr['ccode'].isin(conditions[1])
    else:
        c1 = pr['ccode'].isin(pr['ccode'])
    if conditions[2]:
        c2 = pr['Sub_id'].isin(conditions[2])
    else:
        c2 = pr['Sub_id'].isin(pr['Sub_id'])
    if conditions[3]:
        c3 = pr['type'].isin(conditions[3])
    else:
        c3 = pr['type'].isin(pr['type'])
    if conditions[4]:
        c4 = pr['pgr1'].isin(conditions[4])
    else:
        c4 = pr['pgr1'].isin(pr['pgr1'])
    if conditions[5]:
        if '|' not in conditions[5][0]:
            c5 = pr['ttr'].isin(conditions[5])
        else:
            c5 = pr['ttr'].isin(pr['ttr'])
    else:
        c5 = pr['ttr'].isin(pr['ttr'])
    if conditions[6]:
        c6 = ~pr['ccode'].isin(conditions[6])
    else:
        c6 = pr['ccode'].isin(pr['ccode'])
    if conditions[7]:
        c7 = pr['hip'].isin(conditions[7])
    else:
        c7 = pr['hip'].isin(pr['hip'])
    if conditions[9][0] == 'FMCG':
        pr.loc[c1 & c2 & c3 & c4 & c5 & c6 & c7, 'MCELL_ID_FMCG'] = j['index']
    elif conditions[9][0] == 'CIG':
        pr.loc[c1 & c2 & c3 & c4 & c5 & c6 & c7, 'MCELL_ID_CIG'] = j['index']
    elif conditions[9][0] == 'BEER':
        pr.loc[c1 & c2 & c3 & c4 & c5 & c6 & c7, 'MCELL_ID_BEER'] = j['index']
    elif conditions[9][0] == 'EXXON':
        pr.loc[c1 & c2 & c3 & c4 & c5 & c6 & c7, 'MCELL_ID_EXXON'] = j['index']

In [706]:
# fine tune

pr = pr[pr['MCELL_ID_FMCG'].notna() | pr['MCELL_ID_CIG'].notna() | pr['MCELL_ID_BEER'].notna() | pr['MCELL_ID_EXXON'].notna()]

pr['AP_EXXON'] = pr['AP_FMCG'][pr['MCELL_ID_FMCG'].isna() & pr['MCELL_ID_EXXON'].notna()]
pr.loc[pr['MCELL_ID_FMCG'].isna(), 'AP_FMCG'] = 0

pr.loc[pr['type'].isin(['OPS', 'CDL', 'SPS']) ,'AP_EXXON'] = 0

pr[['AP_FMCG', 'AP_CIG', 'AP_BEER', 'AP_EXXON']] = pr[['AP_FMCG', 'AP_CIG', 'AP_BEER', 'AP_EXXON']].fillna(0)

pr.loc[pr['ccode'] == pr['ops'], 'ops_bool'] = 1
pr['ops_bool'] = pr['ops_bool'].fillna(0)

pr['OP_FMCG'] = 0
pr['OP_CIG'] = 0
pr['OP_BEER'] = 0
pr['OP_EXXON'] = 0

In [707]:
pr.to_excel('HUNTA.xlsx')

In [770]:
pr = pd.read_excel('HUNTA.xlsx')

In [ ]:
#списки для проблемных микроселлов
notinpanel = []
neraskidalos = []

In [793]:
#запустить поочереди для каждого индекса
MCELL_ID = 'MCELL_ID_EXXON'
OP_index = 'OP_EXXON'
AP_index = 'AP_EXXON'

def OP(pr, rules, MCELL_ID, OP_index, AP_index):
    #for i in rules['index'][rules['index'] < 4000].unique():
    #for i in rules['index'][(rules['index'] > 3999) & (rules['index'] < 6000)].unique():
    #for i in rules['index'][(rules['index'] > 5999) & (rules['index'] < 7000)].unique():
    №for i in rules['index'][(rules['index'] > 6999) & (rules['index'] < 8000)].unique():
        if len(pr[pr[MCELL_ID] == i]) != 0:
            OP = rules['OP'][rules['index'] == i].unique().item()
            AP = pr[AP_index][pr[MCELL_ID] == i].sum()
            if (AP != 0) & (OP == 0):
                pass
            elif (AP == 0) & (OP == 0):
                pass
            elif (AP == 0) & (OP != 0):
                #кидаем все в те города, которые являются опсами
                if sum(pr['ops_bool'][pr[MCELL_ID] == i] == 1) > 0:
                    ostatok = OP % len(pr['ops_bool'][pr[MCELL_ID] == i] == 1)
                    OP //= len(pr['ops_bool'][pr[MCELL_ID] == i] == 1)
                    pr.loc[(pr[MCELL_ID] == i) &  (pr['ops_bool'] == 1), OP_index] = OP
                    pr[OP_index][(pr[MCELL_ID] == i) & 
                                  (pr['ops_bool'] == 1) & 
                                  (pr.index == pr.index[(pr[MCELL_ID] == i) & (pr['ops_bool'] == 1)][0])] += ostatok
                #если среди городов нет опсов, распределяем равномерно по всем представленым опсам
                else:
                    while OP != 0:
                        for k in list(pr['ops'][pr[MCELL_ID] == i].unique()):
                            pr[OP_index][(pr[MCELL_ID] == i) & 
                                          (pr['ops'] == k) & 
                                          (pr.index == pr.index[(pr[MCELL_ID] == i) & (pr['ops'] == k)][0])] += 1
                            OP -= 1
                            if OP == 0:
                                break
            elif OP == AP:
                pr.loc[pr[MCELL_ID] == i, OP_index] = pr[AP_index]
            #распеределяем ОП туда, где есть АП, а затем точно также как с : (AP == 0) & (OP != 0)
            elif OP > AP:
                OP -= AP
                pr.loc[(pr[MCELL_ID] == i), OP_index] = pr[AP_index]
                #кидаем все в те города, которые являются опсами
                if sum(pr['ops_bool'][pr[MCELL_ID] == i] == 1) > 0:
                    ostatok = OP % sum(pr['ops_bool'][pr[MCELL_ID] == i] == 1)
                    OP //= sum(pr['ops_bool'][pr[MCELL_ID] == i] == 1)
                    pr[OP_index][(pr[MCELL_ID] == i) & (pr['ops_bool'] == 1)] += OP 
                    pr[OP_index][(pr[MCELL_ID] == i) & 
                                  (pr['ops_bool'] == 1) & 
                                  (pr.index == pr.index[(pr[MCELL_ID] == i) & (pr['ops_bool'] == 1)][0])] += ostatok
                #если среди городов нет опсов, распределяем равномерно по всем представленым опсам
                else:
                    while OP != 0:
                        for k in list(pr['ops'][pr[MCELL_ID] == i].unique()):
                            pr[OP_index][(pr[MCELL_ID] == i) & 
                                          (pr['ops'] == k) & 
                                          (pr.index == pr.index[(pr[MCELL_ID] == i) & (pr['ops'] == k)][0])] += 1
                            OP -= 1
                            if OP == 0:
                                break
            #распеределяем максимум ОП туда, где опс
            elif OP < AP:
                #если АП в опсах == ОП:
                if sum(pr[AP_index][(pr[MCELL_ID] == i) & (pr[AP_index] > 0) & (pr['ops_bool'] == 1)]) == OP:
                    pr.loc[(pr[MCELL_ID] == i) & (pr['ops_bool'] == 1) & (pr[AP_index] > 0), OP_index] = pr[AP_index]
                #если АП в опсах < ОП
                elif sum(pr[AP_index][(pr[MCELL_ID] == i) & (pr[AP_index] > 0) & (pr['ops_bool'] == 1)]) < OP:
                    #заполняем опсы
                    if sum(pr['ops_bool'][pr[MCELL_ID] == i] == 1) > 0:
                        pr.loc[(pr[MCELL_ID] == i) & (pr['ops_bool'] == 1) & (pr[AP_index] > 0), OP_index] = pr[AP_index]
                    #раскидываем ОП по опсам, где есть АП
                    OP -= sum(pr[AP_index][(pr[MCELL_ID] == i) & (pr[AP_index] > 0) & (pr['ops_bool'] == 1)])
                    while OP != 0:
                        for l in list(pr['ccode'][(pr[MCELL_ID] == i) & (pr[AP_index] > 0)].unique()):
                            pr[OP_index][(pr[MCELL_ID] == i) & 
                                        (pr['ccode'] == l) & 
                                        (pr[AP_index] > 0) & 
                                        (pr.index == pr.index[(pr[MCELL_ID] == i) & (pr['ccode'] == l) & (pr[AP_index] > 0)][0])] += 1
                            OP -= 1
                            if OP == 0:
                                break   
                else:
                    cities = list(pr['ccode'][(pr[MCELL_ID] == i) & (pr[AP_index] > 0)].unique())
                    while OP != 0:
                        for l in cities:
                            pr[OP_index][(pr[MCELL_ID] == i) & 
                                          (pr['ccode'] == l) & 
                                          (pr[AP_index] > 0) &
                                          (pr.index == pr.index[(pr[MCELL_ID] == i) & (pr['ccode'] == l) & (pr[AP_index] > 0)][0])] += 1
                            OP -= 1
                            if OP == 0:
                                break
            #микроселлы, где ОП не раскидалась корректно заносятся в этот список
            if rules['OP'][rules['index'] == i].unique().item() != sum(pr[OP_index][pr[MCELL_ID] == i]):
                neraskidalos.append[i]
        #микроселлы, для которых не нашлось куда раскидываться - в этот список
        elif ((len(pr[pr[MCELL_ID] == i]) == 0) & (rules['OP'][rules['index'] == i]) > 0).iloc[0]:
            notinpanel.append(i)
    
    return(pr, notinpanel, neraskidalos)

In [794]:
OP(pr, rules, MCELL_ID, OP_index, AP_index)

C:\Users\ScMa9004\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\ScMa9004\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\ScMa9004\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\ScMa9004\AppD

(       Unnamed: 0  ccode type region_da  pgr1 ttr  hip  Sub_id  \
 0               0      6  DKC   Central     8  TT  NaN      28   
 1               1      6  DKM   Central     8  TT  NaN      28   
 2               2      6  DKP   Central     8  TT  NaN      28   
 3               3      6  FKC   Central     8  TT  NaN      28   
 4               4      6  FKF   Central     8  TT  NaN      28   
 ...           ...    ...  ...       ...   ...  ..  ...     ...   
 13501       13685    437  OSM      Ural     2  TT  IMS      37   
 13502       13686    287  OKS      Ural     5  TT  IMS      40   
 13503       13687    287  OPV      Ural     5  TT  IMS      40   
 13504       13688    467  OSM     Volga     5  TT  IMS      51   
 13505       13689    287  OSM      Ural     5  TT  IMS      40   
 
                          sub  AP_FMCG  ...  CELL_ID_FMCG  CELL_ID_CIG  \
 0      Нижегородская область        3  ...           NaN          NaN   
 1      Нижегородская область        2  ...   

In [822]:
#микроселлы с ненулевой ОП, которых нет в панели - добавить строки
notinpanel

[2680, 2689, 4387, 4406, 4429, 4433, 4504, 4549, 4556, 4820, 5407, 5425, 5517]

In [803]:
#вручную добавить строк для микроселлов которые не раскидались
pr.to_excel('VOZMOZHNO_NE_HUNTA.xlsx')

In [846]:
pr = pd.read_excel('VOZMOZHNO_NE_HUNTA.xlsx')

In [847]:
rules['CELL_ID_FMCG'] = rules['CELL_ID']
rules['CELL_ID_CIG'] = rules['CELL_ID']
rules['CELL_ID_BEER'] = rules['CELL_ID']
rules['CELL_ID_EXXON'] = rules['CELL_ID']
rules['MicroCELL_NAME_FMCG'] = rules['new_mcell_name']
rules['MicroCELL_NAME_CIG'] = rules['new_mcell_name']
rules['MicroCELL_NAME_BEER'] = rules['new_mcell_name']
rules['MicroCELL_NAME_EXXON'] = rules['new_mcell_name']
rules['MCELL_ID_FMCG'] = rules['index']
rules['MCELL_ID_CIG'] = rules['index']
rules['MCELL_ID_BEER'] = rules['index']
rules['MCELL_ID_EXXON'] = rules['index']

In [848]:
pr = pd.merge(pr, rules[['MCELL_ID_FMCG', 'CELL_ID_FMCG', 'MicroCELL_NAME_FMCG']], on = 'MCELL_ID_FMCG', how = 'left')
pr = pd.merge(pr, rules[['MCELL_ID_CIG', 'CELL_ID_CIG', 'MicroCELL_NAME_CIG']], on = 'MCELL_ID_CIG', how = 'left')
pr = pd.merge(pr, rules[['MCELL_ID_BEER', 'CELL_ID_BEER', 'MicroCELL_NAME_BEER']], on = 'MCELL_ID_BEER', how = 'left')
pr = pd.merge(pr, rules[['MCELL_ID_EXXON', 'CELL_ID_EXXON', 'MicroCELL_NAME_EXXON']], on = 'MCELL_ID_EXXON', how = 'left')

In [860]:
pr = pr[['region_normalny', 'region_da', 'Sub_id', 'sub', 'ccode', 'type', 'pgr1', 'ttr', 'hip', 'ops', 'ops_bool', 
         'AP_FMCG', 'OP_FMCG', 'AP_CIG', 'OP_CIG', 'AP_BEER', 'OP_BEER', 'AP_EXXON', 'OP_EXXON', 
         'MCELL_ID_FMCG', 'MCELL_ID_CIG', 'MCELL_ID_BEER', 'MCELL_ID_EXXON', 
         'CELL_ID_FMCG', 'MicroCELL_NAME_FMCG', 'CELL_ID_CIG', 'MicroCELL_NAME_CIG', 'CELL_ID_BEER',
         'MicroCELL_NAME_BEER', 'CELL_ID_EXXON', 'MicroCELL_NAME_EXXON']]

In [861]:
pr.to_excel('NEW_PANEL_RUSSIA_314.xlsx', index = False)